In [ ]:
import sqlalchemy
from urllib import parse
import pymysql
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc 
import matplotlib.font_manager as fm

In [ ]:
import sqlalchemy
from urllib import parse

user = ''
password = ''
host= ''
port = 3306
database = ''
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

con =pymysql.connect(host='', user='', password='', db='', charset='utf8')
cur = con.cursor()

#시군구별 전기차 충전소당 인구수

In [ ]:
#시군구별 전기차 충전소당 인구수
#전기차충전소 = - Table4: `chargeStationCount` : 전국 전기차 충전소 갯수 데이터 `count`             
#인구수 = - Table10: `popInfo` : 전국 총인구조사 데이터 `total_pop`

station_df = pd.read_sql_query("SELECT * FROM chargeStationCount WHERE YEAR ='2023'", con=con)
station_df_sum=pd.DataFrame(station_df.groupby('sido')['count'].sum())

pop = pd.read_sql_query("SELECT sido, sigungu, total_pop FROM popInfo WHERE  sigungu ='합계'", con=con)
change_dict = {'경상북도':"경상", '경상남도':"경상", '부산광역시':"경상", '울산광역시':"경상", '대구광역시':"경상", '전라북도':"전라", '전라남도':"전라", '광주광역시':"전라", '충청북도':"충청", '충청남도':"충청", '세종특별자치시':"충청", '대전광역시':"충청", '전북특별자치도':"전라"}
for key, value in change_dict.items():
  pop['sido'] = pop['sido'].apply(lambda x: value if key in x else x)
pop_sum=pd.DataFrame(pop.groupby('sido')['total_pop'].sum())

poppercharger = pd.concat([station_df_sum,pop_sum],axis=1)

list = []
for x in range(len(poppercharger)):
    list.append(round(poppercharger['total_pop'][x]/poppercharger['count'][x],1))
    #print(round(poppercharger['total_pop'][x]/poppercharger['count'][x],1))
poppercharger['pop/per'] = list





#시군구별 전기차 충전소당 세대수

In [ ]:
#시군구별 전기차 충전소당 세대수
#전기차충전소 = - Table4: `chargeStationCount` : 전국 전기차 충전소 갯수 데이터
#세대수 = - Table10: `popInfo` : 전국 총인구조사 데이터 `total_family`

In [ ]:
family = pd.read_sql_query("SELECT sido, sigungu, total_family FROM popInfo WHERE  sigungu ='합계'", con=con)

change_dict = {'경상북도':"경상", '경상남도':"경상", '부산광역시':"경상", '울산광역시':"경상", '대구광역시':"경상", '전라북도':"전라", '전라남도':"전라", '광주광역시':"전라", '충청북도':"충청", '충청남도':"충청", '세종특별자치시':"충청", '대전광역시':"충청", '전북특별자치도':"전라"}
for key, value in change_dict.items():
  family['sido'] = family['sido'].apply(lambda x: value if key in x else x)

family_sum = pd.DataFrame(family.groupby('sido')['total_family'].sum())
familypercharger = pd.concat([station_df_sum,family_sum],axis=1)

list1 = []
for x in range(len(familypercharger)):
    list1.append(round(familypercharger['total_family'][x]/familypercharger['count'][x],1))

familypercharger['family/per'] = list1




#전체 자동차 수 대비 전기차 수 비율

In [ ]:
#전체자동차 대비 전기차비율
#전체자동차 댓수 -> Table3: `carRegitrectionCount` : 전국 자동차 등록 현황 데이터
#전체전기차 댓수 -> Table6: `electricCarRegitrectionCount`  : 전국 전기차 등록 수 데이터

In [ ]:

carRegitrectionCount = pd.read_sql_query("SELECT SUM(count) FROM carRegitrectionCount WHERE YEAR = '2023' AND MONTH = '12'", con=con)
carRegitrectionCount['SUM(count)'][0]  #=> 25949201
elecratio = round(( electricCarRegitrectionCount['SUM(count)'][0]/carRegitrectionCount['SUM(count)'][0]*100),1) #=> 2.1 

electricCarRegitrectionCount = pd.read_sql_query("SELECT SUM(count) FROM electricCarRegitrectionCount WHERE YEAR = '2023'", con=con)
electricCarRegitrectionCount['SUM(count)'][0]  #=> 538146

difference = carRegitrectionCount['SUM(count)'][0] -electricCarRegitrectionCount['SUM(count)'][0] #=> 25411055 
differenceratio = round(( difference/carRegitrectionCount['SUM(count)'][0]*100),1)



#Matplotlib

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc 
import matplotlib.font_manager as fm
font_path = "./data/NanumGothic.ttf"
font_name = font_manager.FontProperties(fname = font_path).get_name()
plt.rcParams["font.family"] = font_name
f = [f.name for f in fm.fontManager.ttflist]
rc('font', family=font_name)

#시도별 전기차 충전소당 인구수
ratio = [22.7, 880.0, 1196.1, 193.3, 38.7, 145.6, 89.7, 205.3]
labels = ['강원', '경기', '경상', '서울', '인천', '전라', '제주', '충청']

plt.pie(ratio, labels=labels, autopct='%.1f%%')
plt.show()

#시도별 전기차 충전소당 세대수

ratio1 = [11.3, 386.0, 566.6, 92.0, 17.5, 71.0, 41.6, 98.2]
labels1 = ['강원', '경기', '경상', '서울', '인천', '전라', '제주', '충청']

plt.pie(ratio1, labels=labels1, autopct='%.1f%%')
plt.show()

#전체 자동차수 대비 전기차 수 비율
colors = ['gold', 'lightgray' ] 
ratio = [2.1, 97.9]
labels = ['전기차', '화석연료자동차']


plt.pie(ratio, labels=labels, autopct='%.1f%%', colors=colors)
plt.show()colors = ['gold', 'lightgray' ] 
ratio = [2.1, 97.9]
labels = ['전기차', '화석연료자동차']


plt.pie(ratio, labels=labels, autopct='%.1f%%', colors=colors)
plt.show()